In [ ]:
import os
from pprint import pprint
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

from etl.configuration.target_api_config import TargetAPIConfig
from etl.transform.standard_model.graph import ConceptNode
from etl.transform.standard_model.concept_schema import (
    CONCEPT,
    DELIMITER,
    concept_from
)
from etl.transform.standard_model.model import StandardModel
from common.constants import *
from helper import plotly_concept_graph

KIDS_FIRST_CONFIG = '/Users/singhn4/Projects/kids_first/kf-lib-data-ingest/src/target_apis/kids_first.py'

In [ ]:
data = {'pedigree': [{CONCEPT.FAMILY.ID: 'F1',
                 CONCEPT.PARTICIPANT.ID: 'P1',
                 CONCEPT.PARTICIPANT.RACE: RACE.ASIAN},
                {CONCEPT.FAMILY.ID: 'F1',
                 CONCEPT.PARTICIPANT.ID: 'P2',
                CONCEPT.PARTICIPANT.RACE: None}],
'subject_sample': [{CONCEPT.PARTICIPANT.ID: 'P1',
                   CONCEPT.BIOSPECIMEN.ID: 'B1'},
                  {CONCEPT.PARTICIPANT.ID: 'P1',
                   CONCEPT.BIOSPECIMEN.ID: 'B2'},
                  {CONCEPT.PARTICIPANT.ID: 'P2',
                   CONCEPT.BIOSPECIMEN.ID: 'B3'}],
'sample': [{CONCEPT.BIOSPECIMEN.ID: 'B1',
           CONCEPT.PARTICIPANT.RACE: None},
          {CONCEPT.BIOSPECIMEN.ID: 'B2',
           CONCEPT.PARTICIPANT.RACE: None},
          {CONCEPT.BIOSPECIMEN.ID: 'B3',
           CONCEPT.PARTICIPANT.RACE: RACE.WHITE}]}
df_dict = {f's3://bucket/key/{k}.csv': 
           (f'file:///study/configs/{k}.py', pd.DataFrame(v))
      for k, v in data.items()}
    
model = StandardModel()
model.populate(df_dict)
g = model.concept_graph
G = g.graph

In [ ]:
from etl.transform.standard_model.concept_schema import _get_cls_attrs
import json

n = G.nodes['CONCEPT|FAMILY|ID|F1']['object']
_get_cls_attrs(n)

node_dict = ConceptNode.to_dict(n)
n = ConceptNode.from_dict(node_dict)
assert ConceptNode.to_dict(n) == node_dict

def serialize(concept_node):
    if isinstance(concept_node, ConceptNode):        
        n = ConceptNode.to_dict(concept_node)
        return json.dumps(n)
    else:
        return concept_node

def deserialize(node_dict_str):
    d = json.loads(node_dict_str)
    node_obj_dict = ConceptNode.from_dict(d)
    return node_obj_dict

# print("\n".join(nx.generate_gml(G, stringizer=serialize)))

nx.write_gml(G, 'test_graph.gml', stringizer=serialize)

graph = nx.read_gml('test_graph.gml', destringizer=deserialize)

N = graph.node.get('CONCEPT|FAMILY|ID|F1')['object']
N.key
N.uid

In [ ]:
# nx.draw_networkx(g.graph, font_size=10)
plotly_concept_graph(g.graph, notebook_mode=True)

In [ ]:
# Basic graph
graph = {'F1': set(['P1', 'P2']),
         'P1': set(['F1', 'B1', 'B2']),
         'P2': set(['F1', 'B3']),
         'B1': set(['P1', 'White']),
         'B2': set(['P1', 'White']),
         'B3': set(['P2', 'White']),
         'White': set()}
# Build graph
G = nx.DiGraph()
for node, neighbors in graph.items():
    for n in [node] + list(neighbors):
        if not G.has_node:
            G.add_node(n, name=f'Node: {n}')
    for n in neighbors:
        G.add_edge(node, n)
nx.draw_networkx(G, font_size=10)

In [ ]:
nx.write_gml(G, 'test_graph.gml')
graph = nx.read_gml('test_graph.gml')